In [2]:
import pandas as pd
import numpy as np
import torch
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from torcheval.metrics import R2Score
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# %%
df = pd.read_csv("dataset/hf/hf.csv")

perm_cols = ['date', 'PRODUCTREFERENCE']
add_cols = [ 
            # 'ret',
        'aum', 
        'SENT_', 
        # 'PTFSBD', 
        # 'PTFSFX', 
        'PTFSCOM', 
        # 'em', 
        'sp500', 
        'sizespread', 
        'bondmkt', 
        # 'creditspread',
        'SMB', 'HML','RF', 'mom', 
        # 'con','ipg','tfp','term','def','dei','mkt','lab',
        'confeature', 'tfpfeature', 'ipgfeature', 'termfeature', 'deffeature', 'deifeature', 'mktfeature', 'labfeature',
        # 'ret',
        
        ]

target_col = ['exret']

df = df[perm_cols + add_cols + target_col]

# %%
seq_len = 36
pred_len = 1
window_size = seq_len + pred_len

# %%
def generate_date_range(lower, upper, step):

    # List to hold the dates
    date_list = []

    # Generate dates with a 3-month step
    current_date = lower
    while current_date <= upper:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += relativedelta(months=step)
    
    return date_list

# pairs of date start and end bounds for masked time periods
first_start = datetime(2000, 4, 15)
last_start = datetime(2023, 12, 15)
start_list = generate_date_range(first_start, last_start, pred_len)
end_list = generate_date_range(first_start, last_start, pred_len)
pred_start_list = generate_date_range(datetime(2000, 4, 15) - relativedelta(months=seq_len), datetime(2020, 12, 15), pred_len)

print(start_list)

# %%
def create_sliding_windows(data, window_size):
    windows = []
    
    if len(data) < window_size:
        window = data.copy()
        window.reset_index(drop=True, inplace=True)
        windows.append(window)
    else:
        for i in range(len(data) - window_size + 1):
            window = data.iloc[i:i+window_size].copy()
            window.reset_index(drop=True, inplace=True)
            windows.append(window)
    return pd.concat(windows, keys=range(len(windows)))

# %%
scaler = StandardScaler()
df[add_cols] = scaler.fit_transform(df[add_cols])
test_data_df = df.copy()
train_data_df = df.copy().dropna()
pred_data_df = df.copy().dropna()

/home/huopuz/iTransformer-fund_perf/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['2000-04-15', '2000-05-15', '2000-06-15', '2000-07-15', '2000-08-15', '2000-09-15', '2000-10-15', '2000-11-15', '2000-12-15', '2001-01-15', '2001-02-15', '2001-03-15', '2001-04-15', '2001-05-15', '2001-06-15', '2001-07-15', '2001-08-15', '2001-09-15', '2001-10-15', '2001-11-15', '2001-12-15', '2002-01-15', '2002-02-15', '2002-03-15', '2002-04-15', '2002-05-15', '2002-06-15', '2002-07-15', '2002-08-15', '2002-09-15', '2002-10-15', '2002-11-15', '2002-12-15', '2003-01-15', '2003-02-15', '2003-03-15', '2003-04-15', '2003-05-15', '2003-06-15', '2003-07-15', '2003-08-15', '2003-09-15', '2003-10-15', '2003-11-15', '2003-12-15', '2004-01-15', '2004-02-15', '2004-03-15', '2004-04-15', '2004-05-15', '2004-06-15', '2004-07-15', '2004-08-15', '2004-09-15', '2004-10-15', '2004-11-15', '2004-12-15', '2005-01-15', '2005-02-15', '2005-03-15', '2005-04-15', '2005-05-15', '2005-06-15', '2005-07-15', '2005-08-15', '2005-09-15', '2005-10-15', '2005-11-15', '2005-12-15', '2006-01-15', '2006-02-15', '2006

In [5]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()
train_data_df[add_cols] = scaler.fit_transform(train_data_df[add_cols])

In [9]:
train_data_df

,date,PRODUCTREFERENCE,aum,SENT_,PTFSCOM,sp500,sizespread,bondmkt,SMB,HML,...,mom,confeature,tfpfeature,ipgfeature,termfeature,deffeature,deifeature,mktfeature,labfeature,exret
0,2003-03-15,1,-0.170341,-0.498124,-0.431624,0.056465,0.049250,-0.361472,0.168650,-0.652028,...,0.267618,0.014864,-0.029445,0.270889,-0.040977,0.124833,-0.088775,-0.071356,0.424720,0.009900
1,2003-04-15,1,-0.168939,-1.029831,-0.744279,1.764645,0.392897,0.761615,0.302088,0.372637,...,-1.957548,0.034461,-0.031747,0.028382,-0.048623,0.009067,0.038184,-0.010611,0.106670,0.011200
2,2003-05-15,1,-0.163479,-1.047515,1.048727,1.056209,1.885936,-1.765331,1.668931,0.125649,...,-2.239553,-0.005038,0.006577,-0.019463,0.007051,0.011241,0.064202,-0.014796,0.726527,0.012500
3,2003-06-15,1,-0.166075,-0.910677,-1.060332,0.126136,0.140168,-1.063402,0.532901,0.028857,...,-0.242873,0.064155,0.045702,-0.109053,-0.036495,0.002776,-0.018850,-0.061389,0.315260,-0.005700
4,2003-07-15,1,-0.166264,-1.260424,1.046688,0.241324,1.542555,3.101379,1.571557,-0.421729,...,-0.086432,0.011607,-0.028106,0.094174,-0.162277,0.173812,0.061842,-0.053199,0.116954,-0.010900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433788,2019-01-15,107139,-0.053408,-0.333826,-1.303659,1.709190,1.113269,-0.501858,1.026984,-0.161390,...,-1.823750,0.190314,-1.979978,0.190815,-0.930387,-0.735460,-0.168136,1.254790,-0.067045,-0.006100
433789,2019-02-15,107139,-0.059137,-0.260405,-0.392203,0.558705,0.688258,-0.080701,0.561752,-0.899015,...,0.139996,0.001761,-1.285295,0.100240,-0.829668,0.414496,0.033690,0.334517,-0.395244,-0.005800
433790,2019-03-15,107139,-0.060092,-0.183628,0.197083,0.281288,-1.471884,-0.455063,-1.331631,-1.399666,...,0.417884,-0.178491,-1.105097,0.037830,-0.728060,1.064325,-0.755862,0.052260,-0.489492,-0.009900
433964,2017-11-15,107212,0.267492,-0.281619,-0.819724,0.520016,-0.062172,0.012890,-0.184782,-0.017871,...,-0.216113,-0.172141,-0.004999,0.004160,0.080115,0.100632,-0.171755,-0.109110,-0.115476,0.000857


In [28]:
df = pd.read_csv("dataset/hf/hf.csv")

perm_cols = ['date', 'PRODUCTREFERENCE']
add_cols = [ 
            # 'ret',
        'aum', 
        'SENT_', 
        # 'PTFSBD', 
        # 'PTFSFX', 
        'PTFSCOM', 
        # 'em', 
        'sp500', 
        'sizespread', 
        'bondmkt', 
        # 'creditspread',
        'SMB', 'HML','RF', 'mom', 
        # 'con','ipg','tfp','term','def','dei','mkt','lab',
        'confeature', 'tfpfeature', 'ipgfeature', 'termfeature', 'deffeature', 'deifeature', 'mktfeature', 'labfeature',
        # 'ret',
        'exret',
        ]


df = df[perm_cols + add_cols]

In [6]:
seq_len = 36
pred_len = 3
window_size = seq_len + pred_len

In [29]:
def generate_date_range(lower, upper, step):

    # List to hold the dates
    date_list = []

    # Generate dates with a 3-month step
    current_date = lower
    while current_date <= upper:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += relativedelta(months=step)
    
    return date_list

# pairs of date start and end bounds for predictions
start_list = generate_date_range(datetime(2000, 2, 15), datetime(2023, 10, 15), pred_len)
end_list = generate_date_range(datetime(2000, 4, 15), datetime(2023, 12, 15), pred_len)
pred_start_list = generate_date_range(datetime(2000, 2, 15) - relativedelta(months=seq_len), datetime(2020, 10, 15), pred_len)

In [30]:
def create_sliding_windows(data, window_size):
    windows = []
    
    if len(data) < window_size:
        window = data.copy()
        window.reset_index(drop=True, inplace=True)
        windows.append(window)
    else:
        for i in range(len(data) - window_size + 1):
            window = data.iloc[i:i+window_size].copy()
            window.reset_index(drop=True, inplace=True)
            windows.append(window)
    return pd.concat(windows, keys=range(len(windows)))

In [31]:
i = 0

In [34]:
test_data_df = df.copy()
train_data_df = df.copy().dropna()
pred_data_df = df.copy().dropna()

train_df1 = train_data_df[train_data_df.date < start_list[i]]
train_df2 = train_data_df[train_data_df.date > end_list[i] ]
test_df = test_data_df.loc[(test_data_df.date >= start_list[i]) & (test_data_df.date <= end_list[i]), perm_cols + ['exret']]
predict_df = pred_data_df[(pred_data_df.date >= pred_start_list[i]) & (pred_data_df.date < start_list[i])]

# only including funds with full pred len in test df
test_fund_df = test_df.groupby('PRODUCTREFERENCE').agg({'date': 'count'}).reset_index()
test_df = test_df[test_df.PRODUCTREFERENCE.isin(list(test_fund_df[test_fund_df.date >= pred_len]['PRODUCTREFERENCE']))]

# inference performed on only funds present in test and have enough history
count_df = predict_df.groupby(['PRODUCTREFERENCE']).agg({'date': 'count'}).reset_index()
predict_df.merge(count_df.loc[count_df.date >= seq_len], on='PRODUCTREFERENCE') 
funds_to_eval = list(predict_df.merge(test_df, on='PRODUCTREFERENCE', how='inner')['PRODUCTREFERENCE'].unique())
predict_df = predict_df[predict_df.PRODUCTREFERENCE.isin(funds_to_eval)]
predict_df['series_id'] = predict_df['PRODUCTREFERENCE']

# if only including the products with enough history
predict_fund_history_count = predict_df.groupby(['PRODUCTREFERENCE']).agg({'date':'count'}).reset_index()
predict_funds = list(predict_fund_history_count.loc[predict_fund_history_count.date >= seq_len, 'PRODUCTREFERENCE'])
predict_df = predict_df[predict_df.PRODUCTREFERENCE.isin(predict_funds)]

In [36]:
# Step 3: Create sliding windows of size seq len and stack them
window_size = seq_len + pred_len

windows = []

for i in train_df1.PRODUCTREFERENCE.unique():
    temp_df = train_df1[train_df1.PRODUCTREFERENCE == i]
    # if len(temp_df) < 40:
    #     continue
    if len(temp_df) < window_size:
        continue

    sliding_windows = create_sliding_windows(temp_df, window_size)
    sliding_windows = sliding_windows.reset_index()
    sliding_windows['series_id'] = sliding_windows['PRODUCTREFERENCE'].astype("string") + '_0' + '_' +  \
                                   sliding_windows['level_0'].astype("string")
    sliding_windows = sliding_windows.drop(columns=['level_0', 'level_1'])
    windows.append(sliding_windows)

for i in train_df2.PRODUCTREFERENCE.unique():
    temp_df = train_df2[train_df2.PRODUCTREFERENCE == i]
    # if len(temp_df) < 40:
    #     continue
    if len(temp_df) < window_size:
        continue

    sliding_windows = create_sliding_windows(temp_df, window_size)
    sliding_windows = sliding_windows.reset_index()
    sliding_windows['series_id'] = sliding_windows['PRODUCTREFERENCE'].astype("string") + '_1' + '_' +  \
                                   sliding_windows['level_0'].astype("string")
    sliding_windows = sliding_windows.drop(columns=['level_0', 'level_1'])
    windows.append(sliding_windows)

In [37]:
train_input_df = pd.concat(windows).reset_index(drop=True)
train_input_df ['date'] = pd.to_datetime(train_input_df ['date'], format= '%Y-%m-%d' )
train_input_df 

,date,PRODUCTREFERENCE,aum,SENT_,PTFSCOM,sp500,sizespread,bondmkt,SMB,HML,...,confeature,tfpfeature,ipgfeature,termfeature,deffeature,deifeature,mktfeature,labfeature,exret,series_id
0,2003-03-15,1,68531426.00,-0.348188,-0.0651,0.008358,0.002793,-0.0009,0.0065,-0.0193,...,0.000846,-0.000100,0.002948,-0.000016,0.000989,-0.000768,0.000187,0.008730,0.0099,1_1_0
1,2003-04-15,1,70000000.00,-0.656042,-0.1111,0.081044,0.012608,0.0015,0.0102,0.0114,...,0.001122,-0.000133,-0.000119,-0.000117,-0.001021,0.000363,0.001502,0.003320,0.0112,1_1_0
2,2003-05-15,1,75718485.00,-0.666281,0.1527,0.050899,0.055252,-0.0039,0.0481,0.0040,...,0.000567,0.000422,-0.000724,0.000623,-0.000983,0.000595,0.001411,0.013864,0.0125,1_1_0
3,2003-06-15,1,73000000.00,-0.587053,-0.1576,0.011322,0.005390,-0.0024,0.0166,0.0011,...,0.001539,0.000990,-0.001858,0.000044,-0.001130,-0.000145,0.000403,0.006868,-0.0057,1_1_0
4,2003-07-15,1,72801540.49,-0.789554,0.1524,0.016224,0.045444,0.0065,0.0454,-0.0124,...,0.000801,-0.000081,0.000713,-0.001630,0.001840,0.000574,0.000580,0.003495,-0.0109,1_1_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5505079,2022-04-15,106523,2350000.00,1.368416,-0.1253,-0.087957,-0.011568,0.0062,-0.0039,0.0617,...,-0.000235,0.002470,-0.001136,-0.016917,0.001745,0.012820,0.044458,0.002164,0.1142,106523_1_24
5505080,2022-05-15,106523,2400000.00,1.245191,-0.0831,0.000053,-0.000085,0.0015,-0.0006,0.0839,...,0.000000,0.003220,0.000137,-0.027900,0.005355,0.050079,0.002650,0.001828,0.0212,106523_1_24
5505081,2022-06-15,106523,2600000.00,1.177308,0.0158,-0.083920,0.000204,0.0024,0.0130,-0.0597,...,0.001562,0.003685,0.000692,-0.021535,0.003678,0.009342,0.067946,0.001922,0.0770,106523_1_24
5505082,2022-07-15,106523,2430000.00,1.010981,0.0239,0.091116,0.012655,-0.0024,0.0182,-0.0405,...,0.004715,0.002333,-0.005522,-0.012821,0.015545,-0.052806,-0.075792,0.003451,-0.0619,106523_1_24


In [38]:
static_features_df = train_input_df[['series_id', 'PRODUCTREFERENCE']].drop_duplicates().reset_index(drop=True)
predict_static_features = predict_df[['series_id', 'PRODUCTREFERENCE']].drop_duplicates().reset_index(drop=True)


In [39]:
train_data = TimeSeriesDataFrame.from_data_frame(
    train_input_df.drop(columns=['PRODUCTREFERENCE']),
    id_column="series_id",
    timestamp_column="date", 
    static_features_df=static_features_df,
)

predict_data = TimeSeriesDataFrame.from_data_frame(
    predict_df.drop(columns=['PRODUCTREFERENCE']),
    id_column="series_id",
    timestamp_column="date", 
    static_features_df=predict_static_features,
)

predictor = TimeSeriesPredictor(
    prediction_length=3,
    path="benchmark/autogluon-48-3-monthly",
    target="exret",
    eval_metric="MASE",
    freq="MS",
    # verbosity=4,
)

In [40]:
hyperparameters = {
    'TemporalFusionTransformer': {
        # 'context_length': 36  # Fixed look-back window of 5 periods
    },
    # 'RecursiveTabular': {},
    # 'DirectTabular':{},
    # 'Theta':{},
    # 'ETS':{},
}

In [28]:
predictor.fit(
    train_data=train_data,
    hyperparameters=hyperparameters,
    # presets="medium_quality", 
)

Beginning AutoGluon training...
AutoGluon will save models to 'benchmark/autogluon-48-3-monthly'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Mar 29 23:14:13 UTC 2024
CPU Count:          32
GPU Count:          1
Memory Avail:       8.91 GB / 15.48 GB (57.6%)
Disk Space Avail:   69.99 GB / 250.00 GB (28.0%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'freq': 'MS',
 'hyperparameters': {'TemporalFusionTransformer': {}},
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 3,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'exret',
 'verbosity': 2}

train_data with frequency 'None' has been resampled to frequency 'MS'.
Provided train_data has 5890026 rows (NaN fra

In [30]:
predictions = predictor.predict(predict_data)
predictions.head()

data with frequency 'None' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: TemporalFusionTransformer


mean       0.1       0.2       0.3       0.4  \
item_id timestamp                                                      
29      2023-10-01  0.016359 -0.031850 -0.011720  0.000558  0.009764   
        2023-11-01  0.017193 -0.025018 -0.007596  0.003892  0.011263   
        2023-12-01  0.017027 -0.019589 -0.004618  0.005696  0.011875   
35      2023-10-01  0.009378 -0.013475 -0.003925  0.001651  0.006333   
        2023-11-01  0.010172 -0.011307 -0.002161  0.003550  0.007236   

                         0.5       0.6       0.7       0.8       0.9  
item_id timestamp                                                     
29      2023-10-01  0.016359  0.025081  0.036892  0.054590  0.075053  
        2023-11-01  0.017193  0.024977  0.035969  0.052554  0.071784  
        2023-12-01  0.017027  0.024467  0.034189  0.049050  0.067613  
35      2023-10-01  0.009378  0.013495  0.018476  0.026939  0.036781  
        2023-11-01  0.010172  0.013976  0.019346  0.027580  0.037814

In [31]:
pred_df = predictions.reset_index()
pred_df['yearmonth'] = pred_df.timestamp.dt.strftime('%Y-%m')
test_df['yearmonth'] = test_df.date.str.slice(0, -3)
eval_df = pred_df.merge(test_df, how="inner", left_on=['item_id', 'yearmonth'], right_on=['PRODUCTREFERENCE', 'yearmonth'])

In [32]:
preds = list(eval_df['mean'])
trues = list(eval_df['exret'])

metrics = R2Score()
# input = torch.tensor(preds[:,:,0].flatten())
# target = torch.tensor(trues[:,:,0].flatten())

input = torch.tensor(preds)
target = torch.tensor(trues)

metrics.update(input, target)
print(metrics.compute())

tensor(0.0314)


: 